In [1]:
import numpy as np
import os
import time
from PIL import Image
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import keras
from tensorflow.keras import layers, datasets
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

In [3]:
train_img_gen = ImageDataGenerator(fill_mode='nearest',
                                   width_shift_range = 0.15,
                                    height_shift_range = 0.15,
                                    shear_range = 0,
                                    rotation_range = 15,
                                    zoom_range = 0.1,
                                    vertical_flip=False,
                                    horizontal_flip=False
                                                 ,samplewise_center=True
                                                 ,samplewise_std_normalization=True
                                                ) 
val_img_gen = ImageDataGenerator(fill_mode='nearest'
                                                 ,samplewise_center=True
                                                 ,samplewise_std_normalization=True
                                                ) 

In [4]:
def load_data(target_size, batch_size):
    # data path
    train_data_path = r'D:\项目资料\基因表达\数据'
    val_data_path = r'D:\项目资料\基因表达\数据'
    test_data_path = r'D:\项目资料\基因表达\数据'
    
    train_gen = train_img_gen.flow_from_directory(train_data_path,
                                    target_size = target_size,
                                    color_mode = 'rgb',
#                                     classes = [''],
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = None,
                                    interpolation = 'nearest',
                                    keep_aspect_ratio=False
                                                 )
    print('Train data loaded: total: %s, batch size: %s, num classes: %s\n' % (train_gen.n, train_gen.batch_size, train_gen.num_classes))
    
    val_gen = val_img_gen.flow_from_directory(val_data_path,
                                    target_size = target_size,
                                    color_mode = 'rgb',
#                                     classes = [''],
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    shuffle = False,
                                    seed = None,
                                    interpolation = 'nearest',
                                    keep_aspect_ratio=False
                                             )
    print('Val data loaded: total: %s, batch size: %s, num classes: %s\n' % (val_gen.n, val_gen.batch_size, val_gen.num_classes))
    
    return train_gen, val_gen

In [5]:
input_shape=(224, 224, 3)
base_model = tf.keras.applications.resnet50.ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape = input_shape
        )

In [6]:
def build_model():
    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
#     scale_layer = layers.Rescaling(1./255),
    
    base_model.trainable = False
    inputs = keras.Input(shape=input_shape)
    
    # 模型定义
    x = scale_layer(inputs)
#     x = inputs
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Dropout(0.5)(x)  # Regularize with dropout
#     x = tf.keras.layers.Dense(num_classes)(x)
    #     outputs = tf.keras.layers.Softmax()(x)
    outputs = x
    model = tf.keras.Model(inputs, outputs)
    
    model.summary()
    
    return model

In [7]:
model = build_model()
batch_size = 16


# val_img_gen = ImageDataGenerator(fill_mode='nearest'
#                                                  ,samplewise_center=True
#                                                  ,samplewise_std_normalization=True
#                                                 ) 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
# import os
# from PIL import Image
# import numpy as np
# base_path = r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727'
# dirs = [base_path + e for e in [r'\train\M0', r'\train\M1', r'\test\M0', r'\test\M1']]

# exclude_folders = []
# for dir_ in dirs:
#     for file in os.listdir(dir_):
#         file_path = os.path.join(dir_, file)
#         if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
#             exclude_folders.append(file.replace('.txt', ''))

# for dir_ in dirs:
#     for folder in os.listdir(dir_):
#         if folder in exclude_folders:
#             continue
            
#         folder_path = os.path.join(dir_, folder)
#         if os.path.isdir(folder_path):
#             print('folder: %s' % folder)
#             with open(folder_path + '.txt', 'w') as f:
#                 images_path = [os.path.join(folder_path, image) for image in os.listdir(folder_path) \
#                           if image.endswith('.jpg') or image.endswith('.png')]
#                 img_arr = []
#                 for img_path in images_path:
#                     fp = open(img_path, 'rb')
#                     img = Image.open(fp)
#                     img_arr.append(np.array(img))
#                     fp.close()
#                 img_arr = np.array(img_arr)
#                 print(img_arr.shape)
#                 val_gen = val_img_gen.flow(img_arr, batch_size = batch_size, shuffle = False)
#                 results = model.predict(val_gen, batch_size=batch_size, verbose=1)
#                 print(results.shape)
#                 for l in results:
#                     l = [str(e) for e in l]
#                     f.write('%s\n' % ' '.join(l))
                    
#                 val_gen = None
#                 results = None

### 排除目录

In [9]:
import os
from PIL import Image
import numpy as np
png_base_path = r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727'
txt_base_path = r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727特征'
png_dirs = [png_base_path + e for e in [r'\train\M0', r'\train\M1', r'\test\M0', r'\test\M1']]
txt_dirs = [txt_base_path + e for e in [r'\train\M0', r'\train\M1', r'\test\M0', r'\test\M1']]

# exclude_folders = []
# for idx, dir_ in enumerate(png_dirs):
#     for folder in os.listdir(dir_):
#         folder_path = os.path.join(dir_, folder)
#         file_path = os.path.join(txt_dirs[idx], folder + '.txt')
#         if os.path.exists(file_path):
#             png_num = len(os.listdir(folder_path))
#             with open(file_path) as f:
#                 txt_num = len(f.readlines())
#                 print('png num: %s, txt num: %s' % (png_num, txt_num))
#                 if txt_num == png_num:
#                     exclude_folders.append(folder_path)
                    
# print(exclude_folders)

exclude_folders = []
for idx, dir_ in enumerate(txt_dirs):
    for file in os.listdir(dir_):
        file_path = os.path.join(dir_, file)
        folder_path = os.path.join(png_dirs[idx], file.replace('.txt', ''))
        if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
            exclude_folders.append(folder_path)
print(len(exclude_folders))

177


### 图片尺寸不一致的调整

In [24]:
for idx, dir_ in enumerate(png_dirs):
    print('dir: %s' % dir_)
    for folder in os.listdir(dir_):
        folder_path = os.path.join(dir_, folder)
        if folder_path in exclude_folders:
            continue
# folder = os.path.join(r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0', 
#                       'TCGA-CK-5916-01Z-00-DX1.726a78b1-e64f-4dd6-8f7e-e43e98f1f453')
        for img_name in os.listdir(folder_path):
            substr = img_name[-13:]
            if 'w512' not in substr or 'h512' not in substr:
                img_path = os.path.join(folder_path, img_name)
                save_path = os.path.join(folder_path, img_name[:-13] + 'w512-h512.png')
                print(img_path, save_path)
                image = Image.open(img_path)
                image = image.resize((512, 512), Image.Resampling.LANCZOS)
                image.save(save_path)
                os.remove(img_path)
    

dir: \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c63-x31748-y88074-w512-h416.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c63-x31748-y88074-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c91-x46086-y88074-w512-h416.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#t

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c90-x45574-y88074-w512-h416.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c90-x45574-y88074-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c74-x37381-y88074-w512-h416.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770\TCGA-CM-4746-01Z-00-DX1.c83b9795-bf45-4100-9052-a7e485e0f770#tile-r173-c74-x37381-y88074-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangku

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921#tile-r165-c124-x62985-y83980-w512-h448.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921#tile-r165-c124-x62985-y83980-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921#tile-r165-c131-x66570-y83980-w512-h448.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921\TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921#tile-r165-c131-x66570-y83980-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zha

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114#tile-r166-c227-x115729-y84511-w512-h480.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114#tile-r166-c227-x115729-y84511-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114#tile-r166-c240-x122386-y84511-w512-h480.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114\TCGA-CM-6163-01Z-00-DX1.012a7433-73bb-4584-957b-f92c8877a114#tile-r166-c240-x122386-y84511-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010#tile-r109-c345-x176149-y55296-w352-h512.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010#tile-r109-c345-x176149-y55296-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010#tile-r128-c345-x176149-y65024-w352-h512.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010\TCGA-D5-6530-01Z-00-DX1.5c4bbcd1-51ba-467d-93f9-f2a9e7c5e010#tile-r128-c345-x176149-y65024-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据

...

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M0\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4#tile-r3-c191-x97291-y1024-w320-h512.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M0\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4#tile-r3-c191-x97291-y1024-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M0\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4#tile-r96-c79-x39941-y48659-w512-h288.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M0\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4\TCGA-CM-4752-01Z-00-DX1.ac26d5ca-f554-4766-a4c3-f90a8c327dd4#tile-r96-c79-x39941-y48659-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c123-x62464-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c123-x62464-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c89-x45056-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c89-x45056-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhang

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c81-x40960-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c81-x40960-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c74-x37376-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c74-x37376-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangku

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c116-x58880-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c116-x58880-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c121-x61440-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c121-x61440-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zha

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c111-x56320-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c111-x56320-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c124-x62976-y120832-w512-h256.png \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M1\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0\TCGA-AA-A02F-01Z-00-DX1.6E214530-87AE-4E9F-89A9-E35BA9C69BB0#tile-r237-c124-x62976-y120832-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zha

### 删除尺寸为0的图片

In [10]:
import shutil
for idx, dir_ in enumerate(png_dirs):
    print('dir: %s' % dir_)
    for folder in os.listdir(dir_):
        folder_path = os.path.join(dir_, folder)
        if folder_path in exclude_folders:
            continue
# folder = os.path.join(r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0', 
#                       'TCGA-CK-5916-01Z-00-DX1.726a78b1-e64f-4dd6-8f7e-e43e98f1f453')
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            save_path = os.path.join(r'C:\Users\Administrator\Desktop\1', img_name)
            if os.path.getsize(img_path) == 0:
                print(img_path)
                shutil.move(img_path, save_path)


dir: \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CK-5916-01Z-00-DX1.726a78b1-e64f-4dd6-8f7e-e43e98f1f453\TCGA-CK-5916-01Z-00-DX1.726a78b1-e64f-4dd6-8f7e-e43e98f1f453#tile-r138-c69-x34819-y70162-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-6161-01Z-00-DX1.552104aa-6fd7-4d53-918b-fe67d359815c\TCGA-CM-6161-01Z-00-DX1.552104aa-6fd7-4d53-918b-fe67d359815c#tile-r50-c79-x39939-y25092-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-6166-01Z-00-DX1.52eaa124-7ab5-4aaf-b074-7f89a4c53804\TCGA-CM-6166-01Z-00-DX1.52eaa124-7ab5-4aaf-b074-7f89a4c53804#tile-r113-c170-x86537-y57356-w512-h512.png


KeyboardInterrupt: 

In [11]:
for idx, dir_ in enumerate(png_dirs):
    print('dir: %s' % dir_)
    for folder in os.listdir(dir_):
        folder_path = os.path.join(dir_, folder)
        if folder_path in exclude_folders:
            continue

        file_path = os.path.join(txt_dirs[idx], folder + '.txt')
        if os.path.isdir(folder_path):
            print('folder: %s' % folder)
            with open(file_path, 'w') as f:
                ds = tf.keras.utils.image_dataset_from_directory(
                    folder_path,
                    labels=None,
                    label_mode=None,
                    class_names=None,
                    color_mode='rgb',
                    batch_size=batch_size,
                    image_size=input_shape[:2],
                    shuffle=False,
                    interpolation='bilinear'
                )
                results = model.predict(ds, verbose=1)
                print(results.shape)
                for l in results:
                    l = [str(e) for e in l]
                    f.write('%s\n' % ' '.join(l))
                    
                val_gen = None
                results = None

dir: \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0
folder: TCGA-CK-5916-01Z-00-DX1.726a78b1-e64f-4dd6-8f7e-e43e98f1f453
Found 8002 files belonging to 1 classes.
501/501 [==============================] - 131s 252ms/step
(8002, 2048)
folder: TCGA-CM-5349-01Z-00-DX1.d893eb9a-0321-4052-acfc-8c9a6e463921
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 120s 239ms/step
(8000, 2048)
folder: TCGA-CM-5860-01Z-00-DX1.95f23758-00b7-4602-b4ef-944130528f36
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 126s 252ms/step
(8000, 2048)
folder: TCGA-CM-5861-01Z-00-DX1.b900abc0-ecca-48e1-98ba-fbc99a6dae3e
Found 5792 files belonging to 1 classes.
362/362 [==============================] - 89s 247ms/step
(5792, 2048)
folder: TCGA-CM-5863-01Z-00-DX1.2dceed07-9373-4103-be16-533dac9f283b
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 120s 240ms/step
(8000, 2048)
folder: TC

500/500 [==============================] - 128s 255ms/step
(8000, 2048)
folder: TCGA-DM-A0XD-01Z-00-DX1.DAFA56D4-85CB-4FB1-B5BB-E993CA522FF8
Found 881 files belonging to 1 classes.
56/56 [==============================] - 13s 233ms/step
(881, 2048)
folder: TCGA-DM-A0XF-01Z-00-DX1.6FD3D3CF-A1E2-4F4E-BF02-F81B1A1061CC
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 114s 228ms/step
(8000, 2048)
folder: TCGA-DM-A1D0-01Z-00-DX1.1EE92F9A-3DAA-4C1E-9A17-F9E0D31BE0C1
Found 4671 files belonging to 1 classes.
292/292 [==============================] - 75s 257ms/step
(4671, 2048)
folder: TCGA-DM-A1D4-01Z-00-DX1.38346604-2BAF-44F8-BD96-5BF58253C6AD
Found 7028 files belonging to 1 classes.
440/440 [==============================] - 97s 220ms/step
(7028, 2048)
folder: TCGA-DM-A1D6-01Z-00-DX1.BCDA4D6C-8424-477D-9FAF-907206D2DDD6
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 122s 243ms/step
(8000, 2048)
folder: TCGA-DM-A1DA-01Z-

Found 8000 files belonging to 1 classes.
500/500 [==============================] - 123s 245ms/step
(8000, 2048)
folder: TCGA-A6-2671-01Z-00-DX1.13d1a0d9-78cd-4cfc-b670-34a79ebe52ee
Found 2395 files belonging to 1 classes.
150/150 [==============================] - 36s 237ms/step
(2395, 2048)
folder: TCGA-A6-2683-01Z-00-DX1.0dfc5d0a-68f4-45e1-a879-0428313c6dbc
Found 3681 files belonging to 1 classes.
231/231 [==============================] - 49s 212ms/step
(3681, 2048)
folder: TCGA-A6-6648-01Z-00-DX1.88b9a490-0bed-43f3-bd74-1bf2810f6884
Found 1555 files belonging to 1 classes.
98/98 [==============================] - 19s 186ms/step
(1555, 2048)
folder: TCGA-AA-3494-01Z-00-DX1.E275AF20-3AA7-4191-BD1F-FFE744CA6A2F
Found 14 files belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
(14, 2048)
folder: TCGA-AA-3662-01Z-00-DX1.625F1BCC-5E59-411E-AE23-6F43CE6122B2
Found 20 files belonging to 1 classes.
2/2 [==============================] - 1s 97ms/step
(20, 2048)
folder

500/500 [==============================] - 111s 221ms/step
(8000, 2048)
folder: TCGA-NH-A6GC-01Z-00-DX1.29073D7E-5EEF-4BBA-96BE-DC8C69924C42
Found 4380 files belonging to 1 classes.
274/274 [==============================] - 59s 215ms/step
(4380, 2048)
folder: TCGA-NH-A6GC-06Z-00-DX1.5F90CBDB-794B-498D-B75F-60EBDE17B22A
Found 5904 files belonging to 1 classes.
369/369 [==============================] - 82s 220ms/step
(5904, 2048)
folder: TCGA-NH-A8F8-01Z-00-DX1.0C13D583-0BCE-44F7-A4E6-5994FE97B99C
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 115s 230ms/step
(8000, 2048)
dir: \\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\test\M0
folder: TCGA-A6-2672-01Z-00-DX1.e2a845c8-6d77-4120-9f43-abec84a66c1c
Found 2725 files belonging to 1 classes.
171/171 [==============================] - 35s 207ms/step
(2725, 2048)
folder: TCGA-A6-2680-01Z-00-DX1.7b77c0fb-f51d-4d16-ae77-f7615b1d0b87
Found 3929 files belonging to 1 classes.
246/246 [====

500/500 [==============================] - 107s 213ms/step
(8000, 2048)
folder: TCGA-AZ-5407-01Z-00-DX1.5218a617-9817-44f4-8f00-8e9e3d04bd70
Found 4623 files belonging to 1 classes.
289/289 [==============================] - 66s 229ms/step
(4623, 2048)
folder: TCGA-AZ-6608-01Z-00-DX1.40d9f93f-f7d8-4138-9af1-bb579c53194b
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 119s 237ms/step
(8000, 2048)
folder: TCGA-CA-5797-01Z-00-DX1.6549a80e-4b68-4147-949b-6149ab680313
Found 4147 files belonging to 1 classes.
260/260 [==============================] - 57s 220ms/step
(4147, 2048)
folder: TCGA-CK-4947-01Z-00-DX1.b257d5fd-b97b-4987-b088-77f044ca7fe2
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 109s 218ms/step
(8000, 2048)
folder: TCGA-CM-4743-01Z-00-DX1.f54a6355-5623-498c-96b9-2ff1de6576c6
Found 8000 files belonging to 1 classes.
500/500 [==============================] - 108s 216ms/step
(8000, 2048)
folder: TCGA-CM-4752

### 测试异常图片

In [29]:
folder_path = os.path.join(r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0', 
                                    'TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9')
txt_path = os.path.join(r'\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727特征\train\M0', 
                                    'TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9.txt')

results = []
with open(txt_path, 'w') as f:
    images_path = [os.path.join(folder_path, image) for image in os.listdir(folder_path) if image.endswith('.jpg') or image.endswith('.png')]
    for i in range(0, len(images_path), 2):
        img_arr = []
        print(images_path[i])
        fp = open(images_path[i], 'rb')
        img = Image.open(fp)
        img_arr.append(np.array(img))
        fp.close()
        print(images_path[i+1])
        fp = open(images_path[i+1], 'rb')
        img = Image.open(fp)
        img_arr.append(np.array(img))
        fp.close()
        img_arr = np.array(img_arr)
        val_gen = val_img_gen.flow(img_arr, batch_size = 2, shuffle = False)
        results = model.predict(val_gen, batch_size=batch_size, verbose=1)
        for l in results:
            l = [str(e) for e in l]
            f.write('%s\n' % ' '.join(l))
                    
        val_gen = None
        results = None

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r77-c189-x96262-y38916-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r87-c77-x38914-y44036-w512-h512.png
1/1 [==============================] - 1s 923ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r57-c121-x61444-y28675-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r44-c178-x90630-y22018-w5

\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r19-c40-x19969-y9217-w512-h512.png
1/1 [==============================] - 0s 35ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r33-c183-x93190-y16386-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r75-c72-x36354-y37892-w512-h512.png
1/1 [==============================] - 0s 35ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7

1/1 [==============================] - 0s 37ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r45-c76-x38402-y22530-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r64-c68-x34306-y32259-w512-h512.png
1/1 [==============================] - 0s 35ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r47-c57-x28674-y23554-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7

1/1 [==============================] - 0s 36ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r65-c154-x78341-y32771-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r46-c33-x16385-y23042-w512-h512.png
1/1 [==============================] - 0s 34ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r58-c187-x95238-y29187-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41

1/1 [==============================] - 0s 36ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r83-c128-x65028-y41988-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r55-c43-x21505-y27651-w512-h512.png
1/1 [==============================] - 0s 35ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r72-c142-x72197-y36356-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41

...

1/1 [==============================] - 0s 40ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r43-c40-x19969-y21506-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r37-c52-x26114-y18434-w512-h512.png
1/1 [==============================] - 0s 38ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r38-c188-x95750-y18946-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a

1/1 [==============================] - 0s 36ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r41-c138-x70148-y20482-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r33-c37-x18433-y16386-w512-h512.png
1/1 [==============================] - 0s 36ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r40-c59-x29698-y19970-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a

1/1 [==============================] - 0s 38ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r40-c167-x84997-y19970-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r67-c78-x39426-y33795-w512-h512.png
1/1 [==============================] - 0s 37ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r49-c92-x46595-y24578-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a

1/1 [==============================] - 0s 38ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r44-c210-x107015-y22018-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r67-c98-x49667-y33795-w512-h512.png
1/1 [==============================] - 0s 39ms/step
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r66-c133-x67588-y33283-w512-h512.png
\\192.168.0.60\public\模型算法组\训练数据\zhangkuo\基础AI\DX-color-result0727\train\M0\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-4

UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='\\\\192.168.0.60\\public\\模型算法组\\训练数据\\zhangkuo\\基础AI\\DX-color-result0727\\train\\M0\\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9\\TCGA-CM-5348-01Z-00-DX1.2ad0b8f6-684a-41a7-b568-26e97675cce9#tile-r56-c180-x91654-y28163-w512-h512.png'>